# Install Packages

In [34]:
pip install sentence_transformers

# Import packages

In [35]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from datetime import timedelta
import openpyxl
import xlrd
import os

import warnings
warnings.filterwarnings('ignore')

In [36]:
# pytorch library
import torch # the main pytorch library
import torch.nn.functional as f # the sub-library containing different functions for manipulating with tensors

from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

In [37]:
# Code to read csv file into Colaboratory:
from google.colab import drive
drive.mount('/drive')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


In [38]:
pd.options.display.max_colwidth = 10000

# Configs

In [39]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [40]:
path_in_excel = '/drive/MyDrive/Doutorado_v2/Staging/'
path_out_excel = '/drive/MyDrive/Doutorado_v2/Output_Similaridade'

In [41]:
summary_full = pd.read_excel('/drive/MyDrive/Doutorado_v2/E02_file_reportagens.xlsx')
summary_full.head()

,name_file,data_noticia_orig,data_noticia_corrig,Ano,Publication_orig,titulo,Size_notice,Size_notice_cum,Sumarization,Sumarization_Count,text_full
0,3809.pdf,"March 10, 2020 (10:22)",2020-03-10,2020,Jornal O Tempo - Cities,Copam julga recurso da Igreja contra retorno de mineração na Serra da Piedade,77,77,Polêmica se arrasta há mais de um ano; empresa já conseguiu licença prévia,75,b
1,2749.pdf,"June 16, 2020 (03:00)",2020-06-16,2020,Jornal O Tempo - Cities,Justiça dá 15 dias para prefeitura se posicionar sobre mineradora,92,169,Moradores de povoado temem contaminação das águas de nascentes fonte de renda para muitos,90,b
2,3635.pdf,"January 30, 2020 (14:09)",2020-01-30,2020,Zero Hora Field and Crop,Como aproveitar os benefícios do coco durante o verão,97,266,Além de matar a sede a fruta tropical mantém hidratação do corpo e repõe minerais no organismo,95,b
3,2849.pdf,"June 30, 2020 (23:14)",2020-06-30,2020,Jornal O Tempo - Cities,"Barragens da Vale em Minas podem se romper, e famílias serão evacuadas",99,365,Cinco estruturas da mineradora em Ouro Preto e Itabirito estão com cenário extremo de rompimento,97,b
4,3594.pdf,"July 25, 2017",2017-07-25,2017,DCI - News,Mineração,102,467,Órgão anti-fraudes da Inglaterra abriu investigação sobre atuação da mineradora Rio Tinto na Guiné.,100,b


In [42]:
summary_full.shape

(3224, 11)

# Mensure Similarity

In [43]:
a = os.listdir(path_in_excel)
df_files = pd.DataFrame(list(a), columns=['name_file'])
df_files[['a', 'b', 'c']] = df_files['name_file'].str.split('_', 3, expand=True)
df_files[['N', 'e']] = df_files['c'].str.split('.', 2, expand=True)
df_files['files_in'] = df_files['a'] + '_' + df_files['b'] + '_' + df_files['N']
df_files = df_files.drop(['a', 'b', 'c', 'e'], axis=1)
df_files['N'] = df_files['N'].astype(int)
df_files = df_files.sort_values(by=['N'], ascending=[True]).reset_index()
df_files = df_files.drop(['index'], axis=1)

z = (df_files['N'].min()) + 3
df_files = df_files[(df_files['N'] <= z)]

df_files.head(10)

,name_file,N,files_in
0,Similary_Excel_1.xlsx,1,Similary_Excel_1
1,Similary_Excel_2.xlsx,2,Similary_Excel_2
2,Similary_Excel_3.xlsx,3,Similary_Excel_3
3,Similary_Excel_4.xlsx,4,Similary_Excel_4


In [ ]:
%%time
for w in range(0, len(df_files)):

    name_file_in = df_files.iloc[w]['name_file']
    file_excel_in = os.path.join(path_in_excel, name_file_in)
    similary_df = pd.read_excel(file_excel_in)

    for i in range(0, len(similary_df)):

        pdf_col_X = similary_df.iloc[i]['Col_X']
        pdf_col_Y = similary_df.iloc[i]['Col_Y']

        text1 = summary_full[summary_full['name_file'] == pdf_col_X]
        text1 = text1[['Sumarization']].reset_index()
        text11 = text1.loc[0, 'Sumarization']

        text2 = summary_full[summary_full['name_file'] == pdf_col_Y]
        text2 = text2[['Sumarization']].reset_index()
        text22 = text2.loc[0, 'Sumarization']

        texts = [text11, text22]

        texts_embeddings = model.encode(texts)

        a = cosine_similarity([texts_embeddings[0]], texts_embeddings[1:])

        similary_df.loc[i, 'Similarity'] = a[0,0]

    name_file_out = df_files.iloc[w]['name_file']
    file_excel_out = os.path.join(path_out_excel, name_file_out)
    similary_df.to_excel(file_excel_out, index=False)
    print(w)

    os.remove(file_excel_in)
